In [1]:
!pip install albumentations
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... - done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=e9c49904a42a0e417eb08028af48097200157a26b154a841912151b7589d774a
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
Successfully built efficientnet-pytorch


In [2]:
import os
import io
import requests
from PIL import Image
import pandas as pd
import zipfile

In [3]:
path = "../input/hotel-csv/data.csv"
full_df = pd.read_csv(path)
full_df.head()

,image_id,label
0,Balcony/00002170.jpg,Balcony
1,Balcony/00004296.jpg,Balcony
2,Balcony/00002129.jpg,Balcony
3,Balcony/00001760.jpg,Balcony
4,Balcony/00001158.jpg,Balcony


In [4]:
image_dir = "../input/hotel-images/Dataset/"
for i in range(len(full_df)):
    name = full_df.image_id[i]
    full_df.image_id[i] = image_dir + name

In [5]:
full_df.head()

,image_id,label
0,../input/hotel-images/Dataset/Balcony/00002170...,Balcony
1,../input/hotel-images/Dataset/Balcony/00004296...,Balcony
2,../input/hotel-images/Dataset/Balcony/00002129...,Balcony
3,../input/hotel-images/Dataset/Balcony/00001760...,Balcony
4,../input/hotel-images/Dataset/Balcony/00001158...,Balcony


In [6]:
Classes = ["Balcony","Bar","Bathroom","Bedroom","Bussiness Centre","Dining room","Exterior",
           "Gym","Living room","Lobby","Patio","Pool","Restaurant","Sauna","Spa"]

In [7]:
class_to_ind = {}
ind_to_class = {}
for i,cl in enumerate(Classes):
    class_to_ind[cl] = i
    ind_to_class[i] = cl

In [8]:
full_df.label = full_df.label.map(class_to_ind)
full_df.head()

,image_id,label
0,../input/hotel-images/Dataset/Balcony/00002170...,0
1,../input/hotel-images/Dataset/Balcony/00004296...,0
2,../input/hotel-images/Dataset/Balcony/00002129...,0
3,../input/hotel-images/Dataset/Balcony/00001760...,0
4,../input/hotel-images/Dataset/Balcony/00001158...,0


In [9]:
import os
import cv2
from PIL import Image
import pdb
import time
import copy
import warnings
import random
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader, Dataset, sampler
from matplotlib import pyplot as plt
from albumentations.pytorch import ToTensorV2
import albumentations as A
import matplotlib.image as mpi
from pathlib import Path
from sklearn.metrics import recall_score,f1_score
from sklearn.model_selection import StratifiedKFold
import gc
warnings.filterwarnings("ignore")
seed = 53
random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
np.random.seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [10]:
class Dataset(Dataset):
    def __init__(self, df, mean, std, phase):
        self.df = df
        self.mean = mean
        self.std = std
        self.phase = phase
        self.transforms = get_transforms(phase, mean, std)
        self.fnames = self.df.index

    def __getitem__(self, idx):
        # Image data
        
        image_path = self.df['image_id'].iloc[idx]
        img = cv2.imread(image_path)
        img =  cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = self.transforms(image = img)['image']
        
        # Label
        if self.phase == 'test':
            label = 0
        else:
            label = self.df['label'].iloc[idx]
            label = torch.tensor(label, dtype=torch.long)

        inputs = {};
        inputs["images"] = img
        inputs["labels"] = label

        return inputs

    def __len__(self):
        return len(self.fnames)

def get_transforms(phase, mean, std):
    list_transforms = []
    
    if phase == 'train':
        list_transforms.extend(
                  [
                    #A.SmallestMaxSize(max_size=256),
                    A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=10, p=0.5),
                    #A.RandomCrop(height=256, width=256),
                    A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
                    A.RandomBrightnessContrast(p=0.5),
                    A.HorizontalFlip(),
                    A.Rotate(limit=10,p=.5)
                   ]
        )
    list_transforms.extend(
        [ 
            A.Resize(256,256,interpolation = 1),
            A.Normalize(mean=mean, std=std, p=1),
            ToTensorV2(),
        ]
    )
    list_trfms = A.Compose(list_transforms)
    return list_trfms

def provider(
    data_frame,
    phase,
    mean=None,
    std=None,
    batch_size=8,
    num_workers=0,
    split_size = 0.2
):
    '''Returns dataloader for the model training'''
    if phase == "test":
      df = data_frame
    else:
      label = data_frame['label']
      train_df, val_df = train_test_split(data_frame, test_size=split_size,stratify=label)
      df = train_df if phase == "train" else val_df
    
    image_dataset = Dataset(df, mean, std, phase)
    is_train = False if phase == "test" else True
    
    dataloader = DataLoader(
        image_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=False,
        shuffle=is_train,   
    )
    return dataloader

In [11]:
train_data_loader = provider(
                full_df,
                phase='train',
                mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225),
                batch_size=8,
                num_workers=8,
                split_size = 0.2
            )

val_data_loader = provider(
                full_df,
                phase='val',
                mean=(0.485, 0.456, 0.406),
                std=(0.229, 0.224, 0.225),
                batch_size=8,
                num_workers=8,
                split_size = 0.2
            )

In [12]:
import torch.nn as nn
import torch
from efficientnet_pytorch import EfficientNet

In [13]:
class ImgModel(nn. Module):
    def __init__(self, hidden_layer_size,num_classes):
        super().__init__()
        #self.norm1 = nn.BatchNorm2d(3)
        self.effnet = EfficientNet.from_pretrained('efficientnet-b3')
        self._avg_pooling = nn.AdaptiveAvgPool2d(1)
        self.drop = nn.Dropout(0.1)
        self.dense_layer1 = nn.Linear(1536, hidden_layer_size)
        #self.activation = nn.SiLU()
        self.norm1 = nn.BatchNorm1d(hidden_layer_size)
        #self.dense_layer2 = nn.Linear(hidden_layer_size , 256)
        #self.norm2 = nn.BatchNorm1d(256)
        self.out = nn.Linear(256, num_classes)

    def forward(self, inputs):
        
        #eff_out1 = self.norm1(inputs[0])
        eff_out1 = self.effnet.extract_features(inputs['images'])
        eff_out1 = nn.Flatten()(self._avg_pooling(eff_out1))
        eff_out1 = self.drop(eff_out1)
        #self.dense_layer[i].to("cuda")
        
        output = self.dense_layer1(eff_out1)
        output = self.norm1(output)
        #output[0] = self.activation(output[0])
        
        #output = self.dense_layer2(output)
        #output = self.norm2(output)
        
        output = self.out(output)
        output = nn.Softmax()(output)
        
        return output


In [14]:
model = ImgModel(256, 15)
for name,param in model.named_parameters():
    assert(param.requires_grad == True)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b3-5fb5a3c3.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b3-5fb5a3c3.pth


  0%|          | 0.00/47.1M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b3


In [15]:
epochs = 10
batch_size = 8
num_train_steps = int((len(full_df)*0.8) / batch_size * epochs)
#optimizer = AdamW(optimizer_parameters, lr=3e-5)
optimizer_parameters = model.parameters()
optimizer = optim.Adam(optimizer_parameters, lr=3e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer , patience = 5, verbose = True)

In [16]:
model_save_path = '.'

In [17]:
best_loss = np.inf
device = "cuda"
model.to(device)

params = torch.load("../input/effient-b3-hotel/val_loss_1.9051epoch_18.pth")
model.load_state_dict(params['model'])
optimizer.load_state_dict(params['optimizer'])
scheduler.load_state_dict(params['scheduler'])


for epoch in range(epochs):
    model.train()
    train_loss = 0
    num_train_correct = 0
    pbar = tqdm(train_data_loader, total=len(train_data_loader))
    total = 0
    
    for data in pbar:#tqdm(data_loader, total=len(data_loader)):
        data["images"] = data["images"].to(device) 
        
        optimizer.zero_grad()
        op = model(data)
        data["labels"] = data["labels"].to(device)
        loss = nn.CrossEntropyLoss()(op,data["labels"])
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        total += 1
        num_train_correct  += (op.max(1)[1] == data["labels"]).sum().item()
        pbar.set_description(f"loss - {train_loss/total:.6f}, acc - {num_train_correct/(total*batch_size):.6f}")
    
    train_loss = train_loss / len(train_data_loader)
    train_acc = num_train_correct/(len(train_data_loader)*batch_size)
    
    print(f"Train loss  : {train_loss :.6f} , Train_acc : {train_acc:.6f}")
    
    
    
    model.eval()
    eval_loss = 0
    num_eval_correct = 0
    pbar = tqdm(val_data_loader, total=len(val_data_loader))
    total = 0
    
    for data in pbar:#tqdm(data_loader, total=len(data_loader)):
        data["images"] = data["images"].to(device)
        
        op = model(data)
        data["labels"] = data["labels"].to(device)
        loss = nn.CrossEntropyLoss()(op,data["labels"])
        #loss = F1_Loss().cuda()(op,data["labels"])
        eval_loss += loss.item()
        total += 1
        num_eval_correct  += (op.max(1)[1] == data["labels"]).sum().item()
        pbar.set_description(f"loss - {eval_loss/total:.6f}, acc - {num_eval_correct/(total*batch_size):.6f}")
    
    eval_loss = eval_loss / len(val_data_loader)
    eval_acc = num_eval_correct/(len(val_data_loader)*batch_size)
    
    print(f"Val loss  : {eval_loss :.6f} , Val_acc : {eval_acc:.6f}")
    
    scheduler.step(eval_loss)
    
    if eval_loss < best_loss:
      best_loss = eval_loss
      torch.save({ 'model':model.state_dict(), 'optimizer':optimizer.state_dict(),'scheduler':scheduler.state_dict()} ,
                 model_save_path + "/val_loss:" + f"{best_loss:.4f}" + ",epoch:" + str(20+epoch) + ".pth" )
      print("Model saved")

  0%|          | 0/7400 [00:00<?, ?it/s]

Train loss  : 1.902622 , Train_acc : 0.916554


  0%|          | 0/1850 [00:00<?, ?it/s]

Val loss  : 1.902538 , Val_acc : 0.921419
Model saved


  0%|          | 0/7400 [00:00<?, ?it/s]

...

Val loss  : 1.890199 , Val_acc : 0.931216


  0%|          | 0/7400 [00:00<?, ?it/s]

Train loss  : 1.884265 , Train_acc : 0.934307


  0%|          | 0/1850 [00:00<?, ?it/s]

Val loss  : 1.890679 , Val_acc : 0.929324
